In [1]:
import os
import time
import copy
import warnings
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from efficientnet_pytorch import EfficientNet

# Desativa todos os avisos
warnings.filterwarnings("ignore")

# Verificar se há GPU disponível
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Treino usando {device}')

# Definir hiperparâmetros
num_epochs = 30
batch_size = 78
learning_rate = 0.001
data_dir = './images/'  # Substitua pelo caminho para suas pastas de imagens
model_save_path = 'efficientnet_model.pth'

# Transformações de dados
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        transforms.RandomRotation(20),
        transforms.RandomAffine(0, shear=20, scale=(0.8, 1.2)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Carregar os dados
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

# Carregar o modelo EfficientNet pré-treinado
model = EfficientNet.from_pretrained('efficientnet-b0')

# Modificar a última camada para corresponder ao número de classes no seu conjunto de dados
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, len(class_names))

model = model.to(device)

# Definir a função de perda e o otimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Função para treinar o modelo
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)
        
        # Cada época tem uma fase de treino e validação
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Modo de treino
            else:
                model.eval()  # Modo de avaliação

            running_loss = 0.0
            running_corrects = 0

            # Iterar sobre os dados
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zerar os gradientes do otimizador
                optimizer.zero_grad()

                # Forward
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + otimização somente na fase de treino
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Estatísticas
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Salvar o melhor modelo
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    # Carregar os melhores pesos do modelo
    model.load_state_dict(best_model_wts)
    return model

# Treinar o modelo
model = train_model(model, criterion, optimizer, scheduler, num_epochs=num_epochs)

# Salvar o modelo treinado
torch.save(model.state_dict(), model_save_path)
print('Modelo salvo em:', model_save_path)


Treino usando cuda:0
Loaded pretrained weights for efficientnet-b0
Epoch 0/29
----------
train Loss: 3.2692 Acc: 0.2788
val Loss: 1.7642 Acc: 0.5266

Epoch 1/29
----------
train Loss: 1.9085 Acc: 0.5148
val Loss: 1.0385 Acc: 0.6937

Epoch 2/29
----------
train Loss: 1.5563 Acc: 0.5861
val Loss: 0.7798 Acc: 0.7652

Epoch 3/29
----------
train Loss: 1.3577 Acc: 0.6357
val Loss: 0.6395 Acc: 0.8090

Epoch 4/29
----------
train Loss: 1.2268 Acc: 0.6654
val Loss: 0.5897 Acc: 0.8217

Epoch 5/29
----------
train Loss: 1.1166 Acc: 0.6903
val Loss: 0.4441 Acc: 0.8652

Epoch 6/29
----------
train Loss: 1.0365 Acc: 0.7214
val Loss: 0.3780 Acc: 0.8859

Epoch 7/29
----------
train Loss: 0.8289 Acc: 0.7720
val Loss: 0.1948 Acc: 0.9461

Epoch 8/29
----------
train Loss: 0.7136 Acc: 0.8132
val Loss: 0.1630 Acc: 0.9559

Epoch 9/29
----------
train Loss: 0.6727 Acc: 0.8167
val Loss: 0.1477 Acc: 0.9613

Epoch 10/29
----------
train Loss: 0.6635 Acc: 0.8207
val Loss: 0.1352 Acc: 0.9645

Epoch 11/29
-------

In [4]:
import torch
from torchvision import transforms
from PIL import Image
import requests
from io import BytesIO

# Função para processar a imagem
def process_image(image):
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    image = preprocess(image).unsqueeze(0)  # Adiciona uma dimensão para o batch
    return image

# Função para fazer a predição
def predict(image_url, model, class_names):
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content)).convert('RGB')
    image = process_image(image)
    image = image.to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, preds = torch.max(outputs, 1)
        predicted_class = class_names[preds[0]]
    
    return predicted_class

# Caminho para o modelo salvo
model_save_path = 'efficientnet_model.pth'

# Carregar o modelo e as classes
model = EfficientNet.from_name('efficientnet-b0')
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, len(class_names))
model.load_state_dict(torch.load(model_save_path))
model = model.to(device)

In [7]:
# Exemplo de uso
image_url = 'https://d87n9o45kphpy.cloudfront.net/Custom/Content/Products/27/39/2739098_copo-termico-com-abridor-stz-metalizado-preto-5187641_l1_638259794366132214.jpg'  # Substitua pela URL da imagem
predicted_class = predict(image_url, model, class_names)

print('Predicted class:', predicted_class)

Predicted class: copo termico
